In [1]:
from ichs import *

In [2]:
# does state_vector work?
num_qubits = 4
KAK_list = [np.array([np.pi/2, 0, 3*np.pi/2, np.pi/2, 0, np.pi/2, 0,0,0, np.pi/2, -np.pi/2, np.pi, np.pi/2, 3*np.pi/2, 
                      0])] * (num_qubits//2)
init_circ = Ansatz(num_qubits, 1, KAK_list=KAK_list)
state_vector(init_circ.layer_list)

DeviceArray([ 0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
             -1.00000000e+00+2.44929360e-16j,
              0.00000000e+00+0.00000000e+00j,
             -6.12323400e-17+2.24963967e-32j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
             -6.12323400e-17+2.24963967e-32j,
              0.00000000e+00+0.00000000e+00j,
             -3.74939946e-33+1.83667602e-48j], dtype=complex128)

In [13]:
# bigger system
num_qubits = 10
KAK_list = [np.array([np.pi/2, 0, 3*np.pi/2, np.pi/2, 0, np.pi/2, 0,0,0, np.pi/2, -np.pi/2, np.pi, np.pi/2, 3*np.pi/2, 
                      0])] * (num_qubits//2)
init_circ = Ansatz(num_qubits, 1, KAK_list=KAK_list)

step_size = 0.01
operator = trotter_XXZ(num_qubits, step_size, 1)
state_to_match = operator @ state_vector(init_circ.layer_list)
ansatz = Ansatz(num_qubits, 5)

In [14]:
# test all_partial_states
left_state_list, right_state_list = all_partial_states(state_to_match, ansatz.layer_list)
for layer_idx in range(ansatz.num_layers):
    layer_mat = odd_layers(ansatz.layer_list[layer_idx]) if layer_idx%2 else even_layers(ansatz.layer_list[layer_idx])
    print(state_vector(ansatz.layer_list).conj().T @ state_to_match - left_state_list[layer_idx].conj().T @ layer_mat.conj().T @ right_state_list[layer_idx+1])
    
for layer_idx in range(ansatz.num_layers-1,-1,-1): 
    layer_mat = odd_layers(ansatz.layer_list[layer_idx]) if layer_idx%2 else even_layers(ansatz.layer_list[layer_idx])
    print(state_vector(ansatz.layer_list).conj().T @ state_to_match - left_state_list[layer_idx].conj().T @ layer_mat.conj().T @ right_state_list[layer_idx+1])

(2.439454888092385e-18+3.469446951953614e-17j)
(2.2171934427328566e-17+3.8163916471489756e-17j)
(4.640385298237959e-17-1.0408340855860843e-17j)
(2.439454888092385e-17+3.469446951953614e-17j)
(3.5236570605778894e-18+0j)
(3.5236570605778894e-18+0j)
(2.439454888092385e-17+3.469446951953614e-17j)
(4.640385298237959e-17-1.0408340855860843e-17j)
(2.2171934427328566e-17+3.8163916471489756e-17j)
(2.439454888092385e-18+3.469446951953614e-17j)


In [15]:
# test sweep_up_down
for lidx in range(5): 
    new_layer, cost_list = sweep_up_down(left_state_list[lidx], right_state_list[lidx+1], ansatz.layer_list[lidx], lidx, v_sweeps=5)
    print(cost_list[-1])

0.5240679068103866
0.5436423941950808
0.5443893309657398
0.5305756636234069
0.7284225657256973


In [16]:
# test sweep_left_right
ansatz, cost_list = sweep_left_right(state_to_match, ansatz, h_sweeps=2)

0.617283338559284
0.2050082951942349
0.08059776820318343
0.03649535742269405
0.0031897841111522274
0.0031893688477077076
0.002771164128550496
0.002201643146852339
0.0015297126433597708
0.0014426576641090039
0.0014426576641074496
0.0014425564757177822
0.001231593363270611
0.0010370650235790313
0.0008849242575192928
0.0008849241461497126
0.0008239854913553168
0.0007075305949072463
0.0005920758560246098
0.0005846792992171279


In [17]:
1-np.abs(state_vector(ansatz.layer_list).conj().T @ state_to_match)**2

DeviceArray(0.00058468, dtype=float64)

In [18]:
# problem on the second iteration?
layer_list, compression_infidelities, exact_fidelity = ichs(init_circ, 0.1, 0.05, 0.05, 5, h_sweeps=2)

 circuit loop:   0%|                                                                             | 0/2 [00:00<?, ?it/s]

0.8545074469715612
0.6850320915472863
0.4404216819124872
0.13930381865343466
0.011820711122147443
0.01181711494763471
0.00679040064785108
0.004645200631776203
0.0038501280411271654
0.0037827664900830227
0.003782766490084133
0.0037827029178845173
0.00347910921949246
0.003144993120003048
0.002433939909177285
0.0024333021334771443
0.0021661311128777783
0.0019771341098351325
0.0018241802484464387


 circuit loop:  50%|██████████████████████████████████▌                                  | 1/2 [00:36<00:36, 36.53s/it]

0.0018046555935092856
0.04933652911475872
0.01984798783814401
0.010786417168047846
0.009213367826672592
0.005044327877890109
0.005027014230422511
0.004095667240049505
0.003578636181093664
0.003310405523129667
0.003300822520097668
0.003300822520097446
0.0033008140481473047
0.0032455829113869905
0.003091962573796736
0.002564120326754016
0.002557583166716615
0.002322897817953584
0.0021638036070719124
0.002092873982556309


 circuit loop: 100%|█████████████████████████████████████████████████████████████████████| 2/2 [01:12<00:00, 36.18s/it]

0.0020868916127860793


In [20]:
# problem? 
1-exact_fidelity

DeviceArray(0.00780106, dtype=float64)